In [11]:
data = [line.strip().split('\t') for line in open('project3_dataset2.txt', 'r')]

In [12]:
import numpy as np
import math
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)

[['132' '6.20' '6.47' ... '14.14' '45' '0']
 ['123' '0.05' '4.61' ... '2.78' '16' '0']
 ['128' '0.50' '3.70' ... '22.73' '28' '0']
 ...
 ['138' '4.50' '2.85' ... '24.89' '56' '1']
 ['170' '7.60' '5.50' ... '6.17' '54' '1']
 ['128' '0.00' '10.58' ... '14.66' '48' '0']]
[[0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 

In [3]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False

In [13]:
strindex = []
for i in range(len(data[0])):
    if is_categorical(data[0][i]):
        print(data[0][i])
        print(i)
        strindex.append(i)
for i in strindex:
    unique_vals = np.unique(data[:,i])
    mapping = list(range(len(unique_vals)))
    dictionary = dict(zip(unique_vals, mapping))
    for j in range(len(data[:,i])):
        data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

Present
4
[[1.320e+02 6.200e+00 6.470e+00 ... 1.414e+01 4.500e+01 0.000e+00]
 [1.230e+02 5.000e-02 4.610e+00 ... 2.780e+00 1.600e+01 0.000e+00]
 [1.280e+02 5.000e-01 3.700e+00 ... 2.273e+01 2.800e+01 0.000e+00]
 ...
 [1.380e+02 4.500e+00 2.850e+00 ... 2.489e+01 5.600e+01 1.000e+00]
 [1.700e+02 7.600e+00 5.500e+00 ... 6.170e+00 5.400e+01 1.000e+00]
 [1.280e+02 0.000e+00 1.058e+01 ... 1.466e+01 4.800e+01 0.000e+00]]


In [14]:
# for i in range(len(data[:,4])):
#     data[i][4] += 1
    
# print(data[:,4])
print(data.shape)

(462, 10)


In [9]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([j for i,j in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

def calculate_prob(mean,std,test_data):
    prob = test_data[:,0:-1] - mean
    prob = np.multiply(prob,prob)
    prob = (-1 * prob)/(2 * np.multiply(std,std))
    prob = np.exp(prob)
    prob = prob/(((math.pi*2)*std)**0.5)
    prob = np.prod(prob, axis = 1)
    return prob

def remove_categorical(data, strindices):
    tempdata = np.asarray([data[:,i] for i in range(len(data[0])) if i not in strindices])
    tempdata = np.transpose(tempdata)
    return tempdata


In [7]:
def evaluate(actual, predicted):
    #a = TP, b = FN, c= FP, d= TN 
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [15]:
kfoldlist = np.array_split(data, 10)
accuracy = precision = recall = f_measure = 0

for i in range(len(kfoldlist)):
    train, test = split(kfoldlist,i)
    test_labels = []
    label_zeros = []
    label_ones = []
    for j in range(len(train)):
        if(int(train[j,-1]) == 1):
            label_ones.append(train[j,:])
        elif(int(train[j,-1]) == 0):
            label_zeros.append(train[j,:])
    label_zeros = np.asarray(label_zeros)
    label_ones = np.asarray(label_ones)

    # Calculating Prior class probabilites
    prior0 = float(list(train[:,-1]).count(0))/len(train)
    prior1 = float(list(train[:,-1]).count(1))/len(train)
    train_minus_cat = remove_categorical(train, strindex)
    test_minus_cat = remove_categorical(test, strindex)
    mean0 = np.mean([row[0:-1] for row in train_minus_cat if row[-1] == 0],axis = 0)
    mean1 = np.mean([row[0:-1] for row in train_minus_cat if row[-1] == 1], axis = 0)
    std0 = np.std([row[0:-1] for row in train_minus_cat if row[-1] == 0],axis = 0)
    std1 = np.std([row[0:-1] for row in train_minus_cat if row[-1] == 1], axis = 0)
    
    #for categorical attributes
    if(len(strindex) != 0):
        cat_prior0 = dict() 
        cat_prior1 = dict()
        for j in strindex:
            cat_prior0[j] = dict()
            cat_prior1[j] = dict()
            for k in np.unique(train[:,j]):
                nlabels0 = list(label_zeros[:,-1].astype(int)).count(0)
                nlabels1 = list(label_ones[:,-1].astype(int)).count(1)                 
                cat_prior0[j][k] = float(list(label_zeros[:,j]).count(k))/nlabels0
                cat_prior1[j][k] = float(list(label_ones[:,j]).count(k))/nlabels1    
    
    catprob0 = np.zeros(test.shape[0])
    catprob1 = np.zeros(test.shape[0])
    catprob0.fill(1.0)
    catprob1.fill(1.0)
    if(len(strindex) != 0):
        for t in range(len(test)):
            for s in strindex:
                catprob0[t] *= cat_prior0[s][test[t][s]]
                catprob1[t] *= cat_prior1[s][test[t][s]]
                
    gausstest0 = calculate_prob(mean0, std0, test_minus_cat)  
    gausstest1 = calculate_prob(mean1, std1, test_minus_cat) 
    
    #Calculating final probabilities for the test samples
    prob0 = prior0 * np.multiply(gausstest0, catprob0)
    prob1 = prior1 * np.multiply(gausstest1, catprob1)
    test_labels = [1 if prob1[i] > prob0[i] else 0 for i in range(len(test))]
    print(len(test_labels))
    test_labels = np.asarray(test_labels)
    
    a, b, c, d = evaluate(test[:,-1], test_labels)
    accuracy += (float(a + d)/(a + b + c + d))
    precision += (float(a)/(a + c))
    recall += (float(a)/(a + b))
    f_measure += (float(2 * a)/((2 * a) + b + c))

print("Accuracy: ",accuracy * 10,"%")
print("Precision: ",precision * 10,"%")
print("Recall: ",recall * 10,"%")
print("F-1 Measure: ",f_measure * 10,"%")

47
47
46
46
46
46
46
46
46
46
Accuracy:  69.69935245143385 %
Precision:  56.08259193681189 %
Recall:  66.82484474589738 %
F-1 Measure:  60.34466869617516 %
